In [ ]:
def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)
from csv import DictReader

pointcalls_observations_Marseille = []
with open('../../data/navigo_all_pointcalls.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        if row['date_fixed'].split('-')[0] == '1787' \
        and row['pointcall_function'] == "O" \
        and (row["source_suite"] == "Registre du petit cabotage (1786-1787)" \
             or row["source_suite"] == "la Santé registre de patentes de Marseille") \
        and row['net_route_marker'] != "Q":
           # and row['toponyme_fr'] == 'Marseille' \
            pointcalls_observations_Marseille.append(row)

In [ ]:
import pandas as pd


In [ ]:
pointcalls_observations_Marseille = []
with open('../../data/navigo_all_pointcalls.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        if row['date_fixed'].split('-')[0] == '1787' \
        and row['pointcall_function'] == "O" \
        and (row["source_suite"] == "Registre du petit cabotage (1786-1787)" \
             or row["source_suite"] == "la Santé registre de patentes de Marseille") \
        and row['net_route_marker'] != "Q":
           # and row['toponyme_fr'] == 'Marseille' \
            pointcalls_observations_Marseille.append(row)

In [ ]:
all_pointcalls = []
from collections import Counter, defaultdict
with open('../../data/navigo_all_pointcalls.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        if row['date_fixed'].split('-')[0] == '1787' \
        and row["source_suite"] == "G5":
           # and row['toponyme_fr'] == 'Marseille' \
            all_pointcalls.append(row)

In [ ]:
arriere_pays = ["Languedoc", "Corse", "Roussillon", "Provence"]
departures = Counter(row["toponyme_fr"] for row in all_pointcalls if row["pointcall_province"] in arriere_pays)

In [ ]:
departures

In [ ]:
len(pointcalls_entrees_Marseille)

In [ ]:
len(pointcalls_before_Marseille)

In [ ]:
rank_Marseille = {}
for row in pointcalls_observations_Marseille:
    rank_Marseille[row["source_doc_id"]] = row["pointcall_rank_dedieu"]

In [ ]:
ranks_smaller_than_Marseille = []
for row in pointcalls_before_Marseille:
    if row["source_doc_id"] in rank_Marseille:
        max_rank = rank_Marseille[row["source_doc_id"]]
        if row["pointcall_rank_dedieu"] < max_rank:
            ranks_smaller_than_Marseille.append(row)


In [ ]:
polarization_ratios = defaultdict(dict)
for departure, value in departures.items():
    polarization_ratios[departure]["departure_count"] = value
    polarization_ratios[departure]["departure_port"] = departure
    polarization_ratios[departure]["arrival_count"] = 0

for row in ranks_smaller_than_Marseille:
    if row["toponyme_fr"] in departures:
        polarization_ratios[row["toponyme_fr"]]["arrival_count"] += 1



In [ ]:
list_polarisation_ratios = []
for k, v in polarization_ratios.items():
    list_polarisation_ratios.append({"port": k, "value": v["departure_count"], "type": "departure"})
    list_polarisation_ratios.append({"port": k, "value": v["arrival_count"], "type": "arrival"})

In [ ]:
viz_df = pd.DataFrame(list_polarisation_ratios)
viz_df = viz_df.sort_values(["type", "value"])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")

# Draw a nested barplot by species and sex
g = sns.catplot(
    data=viz_df, kind="bar",
    x="value", y="port", hue="type", orient="h",
    height=6
)
g.set_axis_labels("nb de bateaux", "Ports")
plt.title("Nb de bateaux arrivés à Marseille et partis des ports de son bassin commercial")
plt.show()